In [1]:
pip install cirq

In [2]:
#import required libraries
from collections import Counter
import numpy as np
import scipy as sp
import cirq
import time

In [7]:
def make_simon_circuit(input_qubits, output_qubits, secret_string):
    """Solves for the secret period s of a 2-to-1 function such that
    f(x) = f(y) iff x ⨁ y = s
    """
    # Initialise quantum circuit 
    c = cirq.Circuit()

    # Initialize input qubits qubits.
    c.append(
        [
            cirq.H.on_each(*input_qubits),
        ]
    )

    # Query oracle.
    def oracle(input_qubits, output_qubits, secret_string,c):
      for control_qubit, target_qubit in zip(input_qubits, output_qubits):
        c.append(cirq.CNOT(control_qubit, target_qubit))

      # Create mapping:
      if sum(secret_string):  # check if the secret string is non-zero
        significant = list(secret_string).index(1) # Find significant bit of secret string (first non-zero bit)
          
        

      # Add secret string to input according to the significant bit:
      for j in range(len(secret_string)):
        if secret_string[j] > 0:
          c.append(cirq.CNOT(input_qubits[significant], output_qubits[j]))
      
      # Apply a random permutation to create periodicity:
      pos = [
          0,
          len(secret_string) - 1,
          len(secret_string) - 2,
      ] 
      
      # Swap some qubits to define oracle. We choose first, second last and last ones:
      c.append(cirq.SWAP(output_qubits[pos[0]], output_qubits[pos[1]]))

      #if(len(pos)>2):
      #  pass

      c.append(cirq.SWAP(output_qubits[pos[0]], output_qubits[pos[2]]))

    oracle(input_qubits, output_qubits, secret_string,c)
    # Measure in X basis.
    c.append([cirq.H.on_each(*input_qubits), cirq.measure(*input_qubits, key='result')])

    return c


In [8]:
def post_processing(data, results):
    #Solves a system of equations with modulo 2 numbers 
    sing_values = sp.linalg.svdvals(results)
    tolerance = 1e-5
    if sum(sing_values < tolerance) == 0:  # check if measurements are linearly dependent
        flag = True
        null_space = sp.linalg.null_space(results).T[0]
        solution = np.around(null_space, 3)  # ignore very small values
        minval = abs(min(solution[np.nonzero(solution)], key=abs))
        solution = (solution / minval % 2).astype(int)  # renormalize vector mod 2
        data.append(str(solution))
        return flag

In [9]:
def main(qubit_count):

    data = []  # To store results

    # define a secret string:
    secret_string = np.random.randint(2, size=qubit_count)

    print(f'Secret string = {secret_string}')

    n_samples = 100
    for _ in range(n_samples):
        flag = False  # check if we have a linearly independence between our measured data
        while not flag:
            # Choose qubits to use.
            input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]  # input x
            output_qubits = [
                cirq.GridQubit(i + qubit_count, 0) for i in range(qubit_count)
            ]  # output f(x)

            # Pick coefficients for the oracle and create a circuit to query it &
            # Append oracle into special quantum circuit querying it exactly once.
            circuit = make_simon_circuit(input_qubits, output_qubits, secret_string)

            # Sample from the circuit a n-1 times (n = qubit_count).
            simulator = cirq.Simulator()
            results = [
                simulator.run(circuit).measurements['result'][0] for _ in range(qubit_count - 1)
            ]

            # Classical Post-Processing:
            flag = post_processing(data, results)

    freqs = Counter(data)
    print('Circuit:')
    print(circuit)
    print(f'Most common answer was : {freqs.most_common(1)[0]}')

In [ ]:
# Running the algorithm by altering the circuit for varied values of n, where n represents number of qubits & 
# Capturing time taken for the algorithm to run for each value of n
t=[]
for i in range(3,21):
  st=time.time()
  main(i)
  end=time.time()
  t.append(end-st)

Secret string = [0 1 1]
Circuit:
               ┌───┐   ┌──┐
(0, 0): ───H────@───────H─────────────M('result')───
                │                     │
(1, 0): ───H────┼@──────@─────@───H───M─────────────
                ││      │     │       │
(2, 0): ───H────┼┼@─────┼H────┼───────M─────────────
                │││     │     │
(3, 0): ────────X┼┼─────┼─────┼───×───×─────────────
                 ││     │     │   │   │
(4, 0): ─────────X┼─────X─────┼───┼───×─────────────
                  │           │   │
(5, 0): ──────────X───────────X───×─────────────────
               └───┘   └──┘
Most common answer was : ('[0 1 1]', 100)
Secret string = [0 1 0 1]
Circuit:
               ┌────┐   ┌──┐
(0, 0): ───H────@────────H─────────────M('result')───
                │                      │
(1, 0): ───H────┼@───────@─────@───H───M─────────────
                ││       │     │       │
(2, 0): ───H────┼┼@──────┼H────┼───────M─────────────
                │││      │     │       │
(3, 0): ───H──

In [ ]:
import matplotlib.pyplot as plt

# Plotting the time taken by the algorithm against the number of qubits in the circuit to analyze scalability of the algorithm
plt.plot(range(3,21),t)
plt.title('Time vs Bits')
plt.show()
